### Adjust Word Level

In [164]:
#!pip install xlsxwriter

In [39]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


#### While Loop

In [34]:
lang_folder = "Turkish"  # Primary language
lang_pair = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

In [35]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
164294,buldugumuzda,43
164295,boşandıklarını,42
164296,endişenlenme,42
164297,uzaylıymışım,42


In [36]:
#df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence_All.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [37]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_Dutch (nld)_Etymologeek_Result_All.xlsx']

In [38]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,şans,şans,Turkish,(Noun) Fortune. Luck.Fortune. Luck.,dice,Dutch (nld),NaN
1,loca,loca,Turkish,(Noun) (masonic) lodge. Box (in theaters).(mas...,laubja,Dutch (nld),NaN


In [8]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [9]:
def word_count_result(df,column_list): # df is dataframe, bigger_than is integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [10]:
def word_count_bool(df, word_thresh_num, column_list): # df is a dataframe, bigger_than is an integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"count"}, inplace=True)
    
    return  df_word_count["count"][df_word_count.loc[:,"count"] > word_thresh_num].any()  

In [175]:
## while loop code block master
#twogram_num = 100  # 2*step_num minimum: for each word takes two twogram
#twogram_pair_num = 100  # 2*step_num minimum: for each word takes two twogram pair
#word_thresh_num = 7
#
#word_start = 0  # 0
#word_end = 28  # 10
#step_num = word_end  # 10
#word_limit = 28  # 200
#part_num = 1
#while word_end <= word_limit:
#    df_word = df_word_all.iloc[word_start:word_end,]
#    df_word.reset_index(drop=True, inplace=True)
#
#    # language pair twogram
#    ngram_list = []
#    for i in df_pair["dict_entry_main"]:
#        for j in word_tokenize(i):
#            for k in df_word["word"]:
#                twogram_1_2 = f"{j} {k}"
#                ngram_list.append(twogram_1_2)
#                twogram_2_1 = f"{k} {j}"
#                ngram_list.append(twogram_2_1)
#    df_pair_ngram = pd.DataFrame(ngram_list)
#    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
#    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
#    df_pair_ngram.drop_duplicates(inplace=True)
#    df_pair_ngram.reset_index(drop=True, inplace=True)
#    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
#    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
#    df_lang_pair_twogram.drop_duplicates(inplace=True)
#    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
#
#    # twogram
#    word_list = df_word["word"].values.tolist()
#    data_kind = "twogram"
#    twogram_list  = df_twogram_sent.iloc[:,0].values.tolist()
#    
#    resultlist2 = []
#
#    manager = multiprocessing.Manager()
#    resultlist2 = manager.list()
#    
#    def word_in_wordgroup2(list_var2):
#        mergelist = []
#        try:
#            word = list_var2.split()
#        except:
#            pass
#        var1 = range(len(word))
#        for j in var1:
#            if word[j] in word_list:
#                mergelist.append(word[j])
#                if len(mergelist) == len(word):
#                        resultlist2.append(list_var2)
#                            
#    if __name__ == '__main__':
#        # with Pool(16) as p:
#        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
#            p.map(word_in_wordgroup2, twogram_list) # string_word liste 
#
#    result_list2 = list(resultlist2)
#    df_result2 = pd.DataFrame(result_list2)
#    df_result2 = df_result2.rename(columns = {0: f"{data_kind}"})
#    df_result2.iloc[:,0] = df_result2.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
#    df_merge2 = pd.merge(df_result2, df_twogram_sent, how="inner", on=f"{data_kind}")
#    df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
#    df_merge_result2.drop_duplicates(inplace=True)
#    df_merge_result2.reset_index(drop=True, inplace=True)
#    df_twogram_result = df_merge_result2
#    #df_twogram_result = df_twogram_result.head(100)
#
#    # output
#    df_output_result = pd.concat([df_word, df_twogram_result, df_lang_pair_twogram], axis=1)
#
#    df_lesson_result = pd.DataFrame(columns=["word","freq_word","twogram","freq_twogram",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
#    a = 0
#    b = 0
#
#    for i in range(0,110):
#        # Insert words and their count 
#        try:
#            word = df_output_result.iloc[i,0]  # word
#            freq_word = df_output_result.iloc[i,1]  # word freq
#            df_lesson_result.loc[i,"word"] = word
#            df_lesson_result.loc[i,"freq_word"] = freq_word
#        except:
#            pass
#         
#        # Insert n grams
#        try:
#            var1 = df_output_result.iloc[a,2]
#            freq_var1 = df_output_result.iloc[a,3]
#            if (len(df_lesson_result["twogram"]) < twogram_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
#                df_lesson_result.loc[i,"twogram"] = var1
#                df_lesson_result.loc[i,"freq_twogram"] = freq_var1
#                try:
#                    while word_count_bool(df_lesson_result, (word_thresh_num-5), ["twogram"]): # word count result                
#                        a += 1
#                        var1 = df_output_result.iloc[a,2]
#                        freq_var1 = df_output_result.iloc[a,3]
#                        df_lesson_result.loc[i,"twogram"] = var1
#                        df_lesson_result.loc[i,"freq_twogram"] = freq_var1
#                    else:
#                        pass
#                except:
#                    df_lesson_result.loc[i,"twogram"] = np.nan
#                    df_lesson_result.loc[i,"freq_twogram"] = np.nan
#            else:
#                pass
#        except:
#            pass
#        a += 1
#
#        try:
#            var2 = df_output_result.iloc[b,4]
#            freq_var2 = df_output_result.iloc[b,5]
#            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
#                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
#                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
#                try:
#                    while word_count_bool(df_lesson_result, (word_thresh_num-5), [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
#                        b += 1
#                        var2 = df_output_result.iloc[b,4]
#                        freq_var2 = df_output_result.iloc[b,5]
#                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
#                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
#                    else:
#                        pass
#                except:
#                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
#                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
#            else:
#                pass
#        except:
#            pass
#        b += 1
#
#    df_lesson_word_count = word_count_result(df_lesson_result, ["twogram",f"twogram_pair_{lang_pair.lower()}"])
#    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
#    df_lesson_result = df_lesson_result.drop_duplicates()
#    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)
#
#    word_start += step_num
#    word_end += step_num
#    part_num += 1 

#### Condition 1

In [11]:
# while loop code block word and twogram pair
twogram_num = 100  # 2*step_num minimum: for each word takes two twogram
twogram_pair_num = 100  # 2*step_num minimum: for each word takes two twogram pair
word_thresh_num = 2

word_start = 0  # 0
word_end = 7  # 10
step_num = word_end  # 10
word_limit = 28  # 200
part_num = 1
while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    ngram_list = []
    for i in df_pair["dict_entry_main"]:
        for j in word_tokenize(i):
            for k in df_word["word"]:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)

    # output
    df_output_result = pd.concat([df_word, df_lang_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
    a = 0

    for i in range(0,110):
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word 
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert twogram pair
        try:
            var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
            freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, word_thresh_num, [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                        a += 1
                        var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
                        freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

    df_lesson_word_count = word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Set Operation (Test)

In [46]:
part_result_file = glob.glob(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result*.xlsx")
part_result_file

['Turkish_Arabic_28_Word_Step_7_Result1.xlsx',
 'Turkish_Arabic_28_Word_Step_7_Result2.xlsx',
 'Turkish_Arabic_28_Word_Step_7_Result3.xlsx',
 'Turkish_Arabic_28_Word_Step_7_Result4.xlsx']

In [50]:
df_part_all = pd.DataFrame()
for i in part_result_file:
    df_var = pd.read_excel(f"{i}")
    df_part_all = pd.concat([df_part_all,df_var], axis=0)
df_part_twogram_pair = df_part_all.loc[:,[f"twogram_pair_{lang_pair.lower()}"]]
df_part_twogram_pair.reset_index(drop=True, inplace=True)
df_part_twogram_pair

,twogram_pair_arabic
0,bir dakika
1,bir saniye
2,ne kadar
3,bu kadar
4,hayır ben
5,ne yani
6,şey ben
7,ciddi mi
8,bu mükemmel
9,tamam mi


In [55]:
set_part_twogram_pair = set(df_part_twogram_pair[f"twogram_pair_{lang_pair.lower()}"])
set_part_twogram_pair

{'adam da',
 'adam iyi',
 'ama şey',
 'aptal şey',
 'aynı şey',
 'aşk için',
 'bana müsaade',
 'bir dakika',
 'bir saniye',
 'bu kadar',
 'bu mükemmel',
 'bunu asla',
 'ciddi mi',
 'daha basit',
 'daha temiz',
 'dünya kadar',
 'elbette hayır',
 'elbette var',
 'evet elbette',
 'evet tamam',
 'hareket var',
 'hayır ben',
 'hayır değil',
 'hayır lütfen',
 'hayır mı',
 'hayır yok',
 'lütfen ama',
 'lütfen de',
 'merhaba de',
 'mümkün değil',
 'ne kadar',
 'ne yani',
 'rüya gibi',
 'sen ye',
 'seni ahmak',
 'seni aptal',
 'tamam da',
 'tamam iyi',
 'tamam mi',
 'tamam mı',
 've hayır',
 've tekrar',
 'yani bana',
 'yani bunu',
 'yani sen',
 'ye beni',
 'ye kadar',
 'ye onu',
 'yok hayır',
 'çek beni',
 'çek onu',
 'çok garip',
 'çok tuhaf',
 'şey ben',
 'şey gibi',
 'şey için'}

In [58]:
len(set_part_twogram_pair)

56

In [56]:
set_lang_pair_twogram = set(df_lang_pair_twogram[f"twogram_pair_{lang_pair.lower()}"])
set_lang_pair_twogram

{'acaba beni',
 'acaba bunu',
 'acaba onu',
 'acaba seni',
 'acaba yok',
 'adalet yok',
 'adam gibi',
 'adam iyi',
 'adam seni',
 'adam yok',
 'ahmak seni',
 'aile gibi',
 'aile iyi',
 'aile yok',
 'alet yok',
 'alkol yok',
 'aman iyi',
 'amber iyi',
 'aptal gibi',
 'aptal seni',
 'araba gibi',
 'araba iyi',
 'araba yok',
 'asker gibi',
 'asker yok',
 'asla yok',
 'asıl onu',
 'aynı gibi',
 'aziz gibi',
 'aşk gibi',
 'aşk yok',
 'barut gibi',
 'barut yok',
 'basit gibi',
 'beni affetmek',
 'beni asla',
 'beni hala',
 'beni hayır',
 'beni rahat',
 'beni yani',
 'beni ye',
 'beni çek',
 'beni şey',
 'bez yok',
 'bina gibi',
 'budala seni',
 'bunu asla',
 'bunu hayır',
 'bunu kabul',
 'bunu mesela',
 'bunu ye',
 'bunu çek',
 'bunu şey',
 'cahil gibi',
 'cehennem gibi',
 'cehennem iyi',
 'cehennem yok',
 'cenaze gibi',
 'cenaze yok',
 'cennet gibi',
 'cennet yok',
 'ceset gibi',
 'ceset yok',
 'cevap yok',
 'ceza gibi',
 'ceza yok',
 'ciddi gibi',
 'cihaz yok',
 'cin gibi',
 'cuma iyi',
 '

In [59]:
len(set_lang_pair_twogram)

363

In [57]:
set_lang_pair_twogram.difference(set_part_twogram_pair)

{'acaba beni',
 'acaba bunu',
 'acaba onu',
 'acaba seni',
 'acaba yok',
 'adalet yok',
 'adam gibi',
 'adam seni',
 'adam yok',
 'ahmak seni',
 'aile gibi',
 'aile iyi',
 'aile yok',
 'alet yok',
 'alkol yok',
 'aman iyi',
 'amber iyi',
 'aptal gibi',
 'aptal seni',
 'araba gibi',
 'araba iyi',
 'araba yok',
 'asker gibi',
 'asker yok',
 'asla yok',
 'asıl onu',
 'aynı gibi',
 'aziz gibi',
 'aşk gibi',
 'aşk yok',
 'barut gibi',
 'barut yok',
 'basit gibi',
 'beni affetmek',
 'beni asla',
 'beni hala',
 'beni hayır',
 'beni rahat',
 'beni yani',
 'beni ye',
 'beni çek',
 'beni şey',
 'bez yok',
 'bina gibi',
 'budala seni',
 'bunu hayır',
 'bunu kabul',
 'bunu mesela',
 'bunu ye',
 'bunu çek',
 'bunu şey',
 'cahil gibi',
 'cehennem gibi',
 'cehennem iyi',
 'cehennem yok',
 'cenaze gibi',
 'cenaze yok',
 'cennet gibi',
 'cennet yok',
 'ceset gibi',
 'ceset yok',
 'cevap yok',
 'ceza gibi',
 'ceza yok',
 'ciddi gibi',
 'cihaz yok',
 'cin gibi',
 'cuma iyi',
 'daire iyi',
 'ders yok',
 '

In [62]:
df_set_result = pd.DataFrame(set_lang_pair_twogram.difference(set_part_twogram_pair))
df_set_result.rename(columns={0:f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
df_set_result

,twogram_pair_arabic
0,hayır bunu
1,sabun yok
2,cahil gibi
3,veda yok
4,haber yok
...,...
344,maymun yok
345,fare yok
346,fıstık gibi
347,hayır beni


In [60]:
len(set_lang_pair_twogram.difference(set_part_twogram_pair))

349

In [63]:
df_set_pair_twogram = pd.merge(df_lang_pair_twogram, df_set_result, how="inner", on=f"twogram_pair_{lang_pair.lower()}")
df_set_pair_twogram

,twogram_pair_arabic,frequency
0,vakit yok,1366
1,cevap yok,1075
2,elbette yok,1027
3,silah yok,822
4,bunu ye,399
...,...,...
344,halk yok,3
345,cahil gibi,3
346,hatıra yok,3
347,bina gibi,3


#### Condition 2

In [12]:
part_result_file = glob.glob(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result*.xlsx")
part_result_file

['Turkish_Portuguese_28_Word_Step_7_Result1.xlsx',
 'Turkish_Portuguese_28_Word_Step_7_Result2.xlsx',
 'Turkish_Portuguese_28_Word_Step_7_Result3.xlsx',
 'Turkish_Portuguese_28_Word_Step_7_Result4.xlsx']

In [13]:
# Read previous part result
df_part_all = pd.DataFrame()
for i in part_result_file:
    df_var = pd.read_excel(f"{i}")
    df_part_all = pd.concat([df_part_all,df_var], axis=0)
df_part_twogram_pair = df_part_all.loc[:,[f"twogram_pair_{lang_pair.lower()}"]]
df_part_twogram_pair.reset_index(drop=True, inplace=True)
set_part_twogram_pair = set(df_part_twogram_pair[f"twogram_pair_{lang_pair.lower()}"])

# while loop code block
twogram_num = 100  # 2*step_num minimum: for each word takes two twogram
twogram_pair_num = 100  # 2*step_num minimum: for each word takes two twogram pair
word_thresh_num = 7

word_start = 0  # 0
word_end = 28  # 10
step_num = word_end  # 10
word_limit = 28  # 200
part_num = 1
while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    ngram_list = []
    for i in df_pair["dict_entry_main"]:
        for j in word_tokenize(i):
            for k in df_word["word"]:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
    set_lang_pair_twogram = set(df_lang_pair_twogram[f"twogram_pair_{lang_pair.lower()}"])
    df_set_result = pd.DataFrame(set_lang_pair_twogram.difference(set_part_twogram_pair))
    df_set_result.rename(columns={0:f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_set_pair_twogram = pd.merge(df_lang_pair_twogram, df_set_result, how="inner", on=f"twogram_pair_{lang_pair.lower()}")

    # twogram
    word_list = df_word["word"].values.tolist()
    data_kind = "twogram"
    twogram_list  = df_twogram_sent.iloc[:,0].values.tolist()
    
    resultlist2 = []

    manager = multiprocessing.Manager()
    resultlist2 = manager.list()
    
    def word_in_wordgroup2(list_var2):
        mergelist = []
        try:
            word = list_var2.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist2.append(list_var2)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup2, twogram_list) # string_word liste 

    result_list2 = list(resultlist2)
    df_result2 = pd.DataFrame(result_list2)
    df_result2 = df_result2.rename(columns = {0: f"{data_kind}"})
    df_result2.iloc[:,0] = df_result2.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
    df_merge2 = pd.merge(df_result2, df_twogram_sent, how="inner", on=f"{data_kind}")
    df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
    df_merge_result2.drop_duplicates(inplace=True)
    df_merge_result2.reset_index(drop=True, inplace=True)
    df_twogram_result = df_merge_result2
    #df_twogram_result = df_twogram_result.head(100)

    # output
    df_output_result = pd.concat([df_word, df_twogram_result, df_set_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word","twogram","freq_twogram",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
    a = 0
    b = 0

    for i in range(0,110):
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert n grams
        try:
            var1 = df_output_result.iloc[a,2]
            freq_var1 = df_output_result.iloc[a,3]
            if (len(df_lesson_result["twogram"]) < twogram_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram"] = var1
                df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-5), ["twogram"]): # word count result                
                        a += 1
                        var1 = df_output_result.iloc[a,2]
                        freq_var1 = df_output_result.iloc[a,3]
                        df_lesson_result.loc[i,"twogram"] = var1
                        df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

        try:
            var2 = df_output_result.iloc[b,4]
            freq_var2 = df_output_result.iloc[b,5]
            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-5), [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                        b += 1
                        var2 = df_output_result.iloc[b,4]
                        freq_var2 = df_output_result.iloc[b,5]
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
            else:
                pass
        except:
            pass
        b += 1

    df_lesson_word_count = word_count_result(df_lesson_result, ["twogram",f"twogram_pair_{lang_pair.lower()}"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Check Result

In [14]:
df_lesson_result

,word,freq_word,twogram,freq_twogram,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,bir,18835735.0,ne var,62532,harita için,15.0,3.0
1,bu,11062659.0,ben de,59972,bu harita,12.0,3.0
2,ne,8025880.0,değil mi,58386,ve sandviç,7.0,2.0
3,ve,7766036.0,ben mi,33652,bir mango,5.0,3.0
4,için,5484109.0,çok iyi,33627,sandviç var,3.0,2.0
5,mi,5362714.0,ne için,31857,NaN,NaN,2.0
6,ben,4908913.0,hayır değil,18740,NaN,NaN,2.0
7,de,4880315.0,hayır mı,15769,NaN,NaN,2.0
8,çok,4852169.0,bu kadar,15745,NaN,NaN,2.0
9,ama,4661966.0,evet var,11138,NaN,NaN,2.0


In [15]:
word_count_result(df_lesson_result, ["twogram"])

,word,word_count
0,ama,2
1,bana,2
2,çok,2
3,yok,2
4,ve,2
5,var,2
6,seni,2
7,sen,2
8,ne,2
9,mı,2


In [16]:
word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])

,word,word_count
0,harita,2
1,sandviç,2
2,bir,1
3,bu,1
4,için,1
5,mango,1
6,var,1
7,ve,1


#### Read Output File And Multi Write

In [17]:
df_part_11 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result1.xlsx")
df_part_11

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,bir,18835735,bir harita,247.0,2.0
1,bu,11062659,sandviç mi,111.0,1.0
2,ne,8025880,bir sandviç,43.0,NaN
3,ve,7766036,harita bu,17.0,NaN
4,için,5484109,köri mi,13.0,NaN
5,mi,5362714,NaN,NaN,2.0
6,ben,4908913,NaN,NaN,NaN


In [18]:
df_part_12 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result2.xlsx")
df_part_12

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,de,4880315,harita mı,166.0,1.0
1,çok,4852169,evet harita,9.0,NaN
2,ama,4661966,sandviç değil,4.0,NaN
3,var,4389551,sandviç de,3.0,NaN
4,evet,4324786,NaN,NaN,1.0
5,mı,4001316,NaN,NaN,1.0
6,değil,3883885,NaN,NaN,1.0


In [19]:
df_part_13 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result3.xlsx")
df_part_13

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,da,3610161,hayır mango,10.0,NaN
1,şey,3602024,NaN,NaN,NaN
2,hayır,3406992,NaN,NaN,1.0
3,daha,3317577,NaN,NaN,NaN
4,sen,3283654,NaN,NaN,NaN
5,kadar,2697900,NaN,NaN,NaN
6,bana,2659182,NaN,NaN,NaN


In [20]:
df_part_14 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result4.xlsx")
df_part_14

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,yok,2491685,harita yok,25.0,1.0
1,onu,2486889,NaN,NaN,NaN
2,seni,2454988,NaN,NaN,NaN
3,beni,2446696,NaN,NaN,NaN
4,bunu,2445337,NaN,NaN,NaN
5,gibi,2427957,NaN,NaN,NaN
6,iyi,2383224,NaN,NaN,NaN


In [21]:
df_part_21 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_28_Result1.xlsx")
df_part_21

,word,freq_word,twogram,freq_twogram,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,bir,18835735,ne var,62532.0,harita için,15.0,3.0
1,bu,11062659,ben de,59972.0,bu harita,12.0,3.0
2,ne,8025880,değil mi,58386.0,ve sandviç,7.0,2.0
3,ve,7766036,ben mi,33652.0,bir mango,5.0,3.0
4,için,5484109,çok iyi,33627.0,sandviç var,3.0,2.0
5,mi,5362714,ne için,31857.0,NaN,NaN,2.0
6,ben,4908913,hayır değil,18740.0,NaN,NaN,2.0
7,de,4880315,hayır mı,15769.0,NaN,NaN,2.0
8,çok,4852169,bu kadar,15745.0,NaN,NaN,2.0
9,ama,4661966,evet var,11138.0,NaN,NaN,2.0


In [22]:
writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Result_With_Frequency.xlsx", engine='xlsxwriter')

In [23]:
df_part_11.to_excel(writer, sheet_name='Word_Part11', index=False)
df_part_12.to_excel(writer, sheet_name='Word_Part12', index=False)
df_part_13.to_excel(writer, sheet_name='Word_Part13', index=False)
df_part_14.to_excel(writer, sheet_name='Word_Part14', index=False)
df_part_21.to_excel(writer, sheet_name='Word_Part21', index=False)

In [24]:
writer.save()

#### Output File Word Order

In [25]:
df_part_11

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,bir,18835735,bir harita,247.0,2.0
1,bu,11062659,sandviç mi,111.0,1.0
2,ne,8025880,bir sandviç,43.0,NaN
3,ve,7766036,harita bu,17.0,NaN
4,için,5484109,köri mi,13.0,NaN
5,mi,5362714,NaN,NaN,2.0
6,ben,4908913,NaN,NaN,NaN


In [26]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [27]:
word_in_twogram(df_part_11, "word", f"twogram_pair_{lang_pair.lower()}") 

,word,twogram_pair_portuguese
0,bir,bir harita
1,bir,bir sandviç
2,bu,harita bu
3,mi,sandviç mi
4,mi,köri mi


In [28]:
df_word_order_11 = word_in_twogram(df_part_11, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_12 = word_in_twogram(df_part_12, "word", f"twogram_pair_{lang_pair.lower()}") 
df_word_order_13 = word_in_twogram(df_part_13, "word", f"twogram_pair_{lang_pair.lower()}") 
df_word_order_14 = word_in_twogram(df_part_14, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_21 = word_in_twogram(df_part_21, "word", f"twogram") 
df_word_order_212 = word_in_twogram(df_part_21, "word", f"twogram_pair_{lang_pair.lower()}")  

In [29]:
df_word_order_join_11 = df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_12 = df_word_order_12.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_13 = df_word_order_13.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_14 = df_word_order_14.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_211 = df_word_order_21.groupby(["word"])["twogram"].apply(", ".join).reset_index()
df_word_order_join_212 = df_word_order_212.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_21 = pd.merge(df_word_order_join_211, df_word_order_join_212, how="outer", on="word")

In [30]:
df_word_order_join_21

,word,twogram,twogram_pair_portuguese
0,ama,"evet ama, ama yok",NaN
1,bana,"bana mı, bana da",NaN
2,ben,"ben de, ben mi",NaN
3,beni,"seni beni, beni seni",NaN
4,bir,"bir daha, bir şey",bir mango
5,bu,"bu kadar, bu iyi",bu harita
6,bunu,"bunu da, ve bunu",NaN
7,da,"bana da, bunu da",NaN
8,daha,"bir daha, daha çok",NaN
9,de,"ben de, sen de",NaN


In [31]:
writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Join_Result_Without_Frequency.xlsx", engine='xlsxwriter')

In [32]:
df_word_order_join_11.to_excel(writer2, sheet_name='Word_Part11', index=False)
df_word_order_join_12.to_excel(writer2, sheet_name='Word_Part12', index=False)
df_word_order_join_13.to_excel(writer2, sheet_name='Word_Part13', index=False)
df_word_order_join_14.to_excel(writer2, sheet_name='Word_Part14', index=False)
df_word_order_join_21.to_excel(writer2, sheet_name='Word_Part21', index=False)

In [33]:
writer2.save()

#### Youtube Sentence Test

In [1]:
#word_num = 300
#df_word_youtube = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Word Tokenize Merge/Youtube_Word_Tokenize_Merge.csv")
#df_word_select = df_word_youtube.head(word_num)
#df_word_select

In [40]:
df_sent_youtube = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_sent_youtube

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [41]:
df_sent_youtube['start_time'] = pd.to_timedelta(df_sent_youtube['start_time']) # data type converted timedelta for second 
df_sent_youtube['end_time'] = pd.to_timedelta(df_sent_youtube['end_time'])

In [42]:
df_sent_youtube['start_time'] = df_sent_youtube['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_sent_youtube['end_time'] = df_sent_youtube['end_time'].apply(lambda x: x.total_seconds())
df_sent_youtube

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3036932,471.970,472.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,472.470,482.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,482.498,484.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,484.178,488.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [43]:
#word_list = df_word_select.iloc[:,0].to_list()
word_list = ["her şey yolunda"]

In [44]:
df_word_result = pd.DataFrame()
for i in word_list:
    try:
        word_in_video = df_sent_youtube[df_sent_youtube.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]
        #word_in_video = df_sent_youtube[df_sent_youtube.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].sample(200) # try and except will try for sample
    except:
        pass
        #word_in_video = df_sent_youtube[df_sent_youtube.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(200)         
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

,word,start_time,end_time,sentence,video_id
0,her şey yolunda,67.294,72.004,nasıl gidiyor o iyi her şey yolunda,ud9s8aerIJE
1,her şey yolunda,332.360,333.780,sevgi her şey yolunda mı,OPfevmWgETE
2,her şey yolunda,460.320,462.860,sıkıntı yok gayet iyi her şey yolunda,CNPzcVvZ1nU
3,her şey yolunda,1384.160,1387.380,mamasını da yemiş her şey yolunda dün serum al...,GjlduZjqHxA
4,her şey yolunda,1572.000,1573.920,ateşi yok her şey yolunda değil mi,k1ni27c9C2k
...,...,...,...,...,...
395,her şey yolunda,2927.180,2930.161,onu merak etme her şey yolunda hiçbir şeyden h...,v8eY_Xa5SGw
396,her şey yolunda,454.370,455.640,her şey yolunda,axyvWKQ4MQw
397,her şey yolunda,1073.550,1075.852,yok canım bir şey olduğu yok her şey yolunda,psNiD0Ks_uk
398,her şey yolunda,3824.874,3826.462,her şey yolunda,psNiD0Ks_uk


In [45]:
word_time_loc_list = []
def word_time_loc(df):
    for i in range(len(df)):
        word = df.loc[i,"word"]
        start_time = df.loc[i,"start_time"]
        end_time = df.loc[i,"end_time"]
        sentence = df.loc[i,"sentence"]
        video_id = df.loc[i,"video_id"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=["word","start_time","end_time","sentence","video_id"])

    return df_word_time_loc

In [46]:
df_word_result = word_time_loc(df_word_result)
df_word_result

,word,start_time,end_time,sentence,video_id
0,her şey yolunda,69.850857,72.004000,nasıl gidiyor o iyi her şey yolunda,ud9s8aerIJE
1,her şey yolunda,332.655833,333.661667,sevgi her şey yolunda mı,OPfevmWgETE
2,her şey yolunda,461.761622,462.860000,sıkıntı yok gayet iyi her şey yolunda,CNPzcVvZ1nU
3,her şey yolunda,1385.212692,1386.265385,mamasını da yemiş her şey yolunda dün serum al...,GjlduZjqHxA
4,her şey yolunda,1572.508235,1573.468235,ateşi yok her şey yolunda değil mi,k1ni27c9C2k
...,...,...,...,...,...
395,her şey yolunda,2927.938800,2928.860200,onu merak etme her şey yolunda hiçbir şeyden h...,v8eY_Xa5SGw
396,her şey yolunda,454.370000,455.640000,her şey yolunda,axyvWKQ4MQw
397,her şey yolunda,1075.014909,1075.852000,yok canım bir şey olduğu yok her şey yolunda,psNiD0Ks_uk
398,her şey yolunda,3824.874000,3826.462000,her şey yolunda,psNiD0Ks_uk


In [47]:
time_shift = 0.5
df_word_result.start_time = df_word_result.start_time.apply(lambda x: (x-time_shift))
df_word_result.end_time = df_word_result.end_time.apply(lambda x: (x+time_shift))
df_word_result

,word,start_time,end_time,sentence,video_id
0,her şey yolunda,69.350857,72.504000,nasıl gidiyor o iyi her şey yolunda,ud9s8aerIJE
1,her şey yolunda,332.155833,334.161667,sevgi her şey yolunda mı,OPfevmWgETE
2,her şey yolunda,461.261622,463.360000,sıkıntı yok gayet iyi her şey yolunda,CNPzcVvZ1nU
3,her şey yolunda,1384.712692,1386.765385,mamasını da yemiş her şey yolunda dün serum al...,GjlduZjqHxA
4,her şey yolunda,1572.008235,1573.968235,ateşi yok her şey yolunda değil mi,k1ni27c9C2k
...,...,...,...,...,...
395,her şey yolunda,2927.438800,2929.360200,onu merak etme her şey yolunda hiçbir şeyden h...,v8eY_Xa5SGw
396,her şey yolunda,453.870000,456.140000,her şey yolunda,axyvWKQ4MQw
397,her şey yolunda,1074.514909,1076.352000,yok canım bir şey olduğu yok her şey yolunda,psNiD0Ks_uk
398,her şey yolunda,3824.374000,3826.962000,her şey yolunda,psNiD0Ks_uk


In [48]:
df_word_result.start_time = df_word_result.start_time.apply(lambda x: round(x))
df_word_result.end_time = df_word_result.end_time.apply(lambda x: round(x))
df_word_result 

,word,start_time,end_time,sentence,video_id
0,her şey yolunda,69,73,nasıl gidiyor o iyi her şey yolunda,ud9s8aerIJE
1,her şey yolunda,332,334,sevgi her şey yolunda mı,OPfevmWgETE
2,her şey yolunda,461,463,sıkıntı yok gayet iyi her şey yolunda,CNPzcVvZ1nU
3,her şey yolunda,1385,1387,mamasını da yemiş her şey yolunda dün serum al...,GjlduZjqHxA
4,her şey yolunda,1572,1574,ateşi yok her şey yolunda değil mi,k1ni27c9C2k
...,...,...,...,...,...
395,her şey yolunda,2927,2929,onu merak etme her şey yolunda hiçbir şeyden h...,v8eY_Xa5SGw
396,her şey yolunda,454,456,her şey yolunda,axyvWKQ4MQw
397,her şey yolunda,1075,1076,yok canım bir şey olduğu yok her şey yolunda,psNiD0Ks_uk
398,her şey yolunda,3824,3827,her şey yolunda,psNiD0Ks_uk


In [ ]:
#df_word_result.to_excel("Youtube_Sentence_Select_Word_Time_Interval.xlsx", index=False)

In [69]:
df_word_result.start_time.to_list()

[160, 145, 1510, 1000, 539, 143, 793, 277, 977, 627]

In [70]:
df_word_result.end_time.to_list()

[163, 148, 1512, 1001, 551, 153, 798, 281, 978, 628]

In [14]:
df_word_result.video_id.to_list()

['A4wf93zJZIQ',
 'gOq59m1pUi4',
 'ifxg2zkaN5E',
 'yKx4zO8_Jhk',
 'CNPzcVvZ1nU',
 'GWzi144zrEs',
 'GWzi144zrEs',
 '6FtA8_D7Ynw',
 'M5zJ_X8i3rI',
 'WYcGfSiTbYY']

In [71]:
",".join(df_word_result.video_id.to_list())

'c9ymkFAiZDk,BsRzeONLa28,dWIJFBbJSn4,GhMqYDwawUU,A4wf93zJZIQ,gOq59m1pUi4,vOWbfQ0tIyU,uzYXX4RgcO0,6FtA8_D7Ynw,F7OQWEJJR34'

#### Not Twogram Sent In Sent Threegram

In [4]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence_All.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext.
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [5]:
df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [6]:
set_twogram_sent = set(df_twogram_sent.twogram)
set_twogram = set(df_twogram.twogram)

In [7]:
df_ngram = pd.DataFrame(set_twogram.difference(set_twogram_sent))
df_ngram

,0
0,olunca insanlar
1,fikret bilgisayarı
2,oy toplama
3,bunu anlatmalıydın
4,el kitabının
...,...
4171721,eliyle çalışıyor
4171722,daima güvenebileceğim
4171723,pariste yaptırmış
4171724,i̇kâmet belgemi


In [8]:
n_gram_list = df_ngram.iloc[:,0].to_list()
n_gram_list

['olunca insanlar',
 'fikret bilgisayarı',
 'oy toplama',
 'bunu anlatmalıydın',
 'el kitabının',
 've istenmeyen',
 'ey garip',
 'edeceğini şaşırmıştı',
 'peşinde brill',
 'hani size',
 'minseok babasız',
 'evdeki sürtüğe',
 'belicoff bana',
 'olası karın',
 'kilit de',
 'tozu getirdim',
 'mı beklemeyi',
 'uçuş united',
 'konuşabilen bir',
 'lima peruya',
 'xavier slaughtery',
 'çayı burada',
 'isyanlar ve',
 'gözünden kaçmayacak',
 'noktalama işaretiyle',
 '222 ve',
 'zamanında yetişirse',
 'gülerim onlara',
 'güldüren adamları',
 'hediyesini kira',
 'seni tanımadan',
 'çekini kontrol',
 'karı bul',
 'yüzyılın ilişkisiydi',
 'sadece bolonez',
 'ben cinayettenim',
 'bay tamnusı',
 'bozuyordu çünkü',
 'malzeme beklemiyoruz',
 'araba satman',
 'edilmesine kalan',
 'oraya hissettiğim',
 'chrysler fabrikasında',
 'bekleyebilirim diye',
 'adım arkamda',
 'sürücü öğretmenisin',
 'bunu karşılayacak',
 'olduğu kanıtlanamadı',
 'hiromichi horikawa',
 'önce duymamışsınız',
 'istersem istediğim'

In [9]:
df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_n_sent = df_threegram

In [10]:
result_df = []

In [ ]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_df = manager.list()

def wordgroup_in_wordgroup(n_gram_list):
    df_wordgroup_result = pd.DataFrame()
    df_result = df_n_sent[df_n_sent.iloc[:,0].str.contains(fr"(^|\s){n_gram_list}(\s|$)")]
    df_result.insert(0,f"all_ngram",n_gram_list)
    df_wordgroup_result = pd.concat([df_wordgroup_result,df_result], axis=0)
    df_wordgroup_result.reset_index(drop=True, inplace=True)
    result_df.append(df_wordgroup_result)    
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(wordgroup_in_wordgroup, n_gram_list) # string_word liste

In [ ]:
multi_df_list = list(result_df)
df_concat = pd.DataFrame()
for i in multi_df_list:
    df_concat = pd.concat([df_concat,i],axis=0)
df_concat.sort_values(by="frequency",ascending=False, inplace=True)
df_concat.reset_index(drop=True, inplace=True)
df_concat

In [ ]:
df_concat.to_excel("Not_Twogram_Sent_In_Threegram_Sent.xlsx", index=False)